# Data Exploration - Data Analysis
# Goals of this notebook

* Create direct access to data storage in the XM API
* Process the information to generate tables and dynamic inputs
* Produce an object/product that compiles the developed analysis

## Index

1. [Importing libraries](#section1)
2. [Queries to databases via API](#section2)
3. [Data processing into formats](#section3)
4. [Production of an object with processing results(#section4)
ults](#section4)
n4)

<a id='section1'></a>
# 1. Importing libraries
Several libraries must be imported to enable queries to data repositories, perform data processing, and support the development of the resulting dashboard.

In [83]:
# In the command prompt console, you must install and run these commands
# pip install -r requirements.txt

# Datetime libraries
import datetime as dt
import time
from datetime import datetime, timedelta
import calendar
import pytz
from babel.dates import format_date

# Data processing libraries
import pandas as pd
import numpy as np
from functools import reduce

# XM libraries
from pydataxm import * 

<a id='section2'></a>
# 2. Querying Databases via API

XM provides an API within its Sinergox information publishing platform. This tool allows users to create queries to extract relevant information from the Colombian Wholesale Energy Market. On GitHub, the [API XM](https://github.com/EquipoAnaliticaXM/API_XM) repository contains a guide on how to extract the information, but a brief explanation will be provided below. It is important to note that using the XM API does not require managing any user credentials or keys, which makes access easier.

## API Queries

Each data query initially requires requesting the information with the following code, which subsequently displays the variables stored in XM’s public erver:


In [2]:
objetoAPI = pydataxm.ReadDB()
df =objetoAPI.get_collections() 
df.MetricId.unique()           

array(['DemaReal', 'ExpoMoneda', 'DemaCome', 'Gene', 'GeneIdea',
       'PrecBolsNaci', 'RestAliv', 'ConsCombustibleMBTU', 'IndRecMargina',
       'PrecOferDesp', 'MaxPrecOferNal', 'CompBolsNaciEner',
       'CompContEner', 'VentContEner', 'ConsCombAprox', 'EmisionesCO2',
       'EmisionesCH4', 'EmisionesN2O', 'EmisionesCO2Eq',
       'factorEmisionCO2e', 'ImpoEner', 'ImpoMoneda', 'ExpoEner',
       'DemaOR', 'PerdidasEner', 'GeneSeguridad', 'GeneFueraMerito',
       'GeneProgDesp', 'GeneProgRedesp', 'DispoReal', 'DispoCome',
       'DispoDeclarada', 'RecoPosEner', 'RecoNegEner', 'RespComerAGC',
       'DesvEner', 'DesvMoneda', 'RecoPosMoneda', 'RecoNegMoneda',
       'ExcedenteAGPE', 'DemaComeReg', 'DemaComeNoReg', 'RestSinAliv',
       'PrecPromContNoRegu', 'PrecPromContRegu', 'CompContEnerReg',
       'CompContEnerNoReg', 'RentasCongestRestr', 'EjecGarantRestr',
       'SnTIEMerito', 'SnTIEFueraMerito', 'PrecBolsNaciTX1',
       'CompBolsaTIEEner', 'CompBolsaIntEner', 'VentBolsaTIEE

Next, we present the method to query the cross-references for a specific metric.  
You must enter the **MetricId** of the indicated variable


In [4]:
df =objetoAPI.get_collections("Gene")
df

,MetricId,MetricName,Entity,MaxDays,Type,Url,Filter,MetricUnits,MetricDescription
4,Gene,Generación por Sistema,Sistema,31,HourlyEntities,https://servapibi.xm.com.co/hourly,No aplica,kWh,Generacion neta de cada una de las plantas Nac...
10,Gene,Generación por Recurso,Recurso,31,HourlyEntities,https://servapibi.xm.com.co/hourly,Codigo Submercado Generación,kWh,Generacion neta de cada una de las plantas Nac...


As can be seen, a DataFrame is generated with the data structure available for the queried variable. In this case, the columns contain the following information:

- **MetricID:** Unique identifier of the metric to be queried  
- **MetricName:** Full name of the MetricID identifier  
- **Entity:** Grouping parameter of the metric  
- **MaxDays:** Maximum number of days available in the query  
- **Type:** Granularity with which the data is reported  
- **Url:** Endpoint of the data query  
- **Filter:** Filter by query parameter  
- **MetricUnits:** Unit of measurement of the metric  
- **MetricDescription:** Description of the query parameter

## Downloading Required Databases

In this step, all the necessary datasets for the exercise are downloaded and stored in the **Project Folder** named **Datasets**. This ensures that the information is readily available for anyone that has problems accesing XM's API.

In [37]:
# Query generation data
pofe = objetoAPI.request_data(
    "PrecOferDesp",        # Specify the name of the metric as defined in the MetricID field
    "Recurso",             # Specify the name of the entity as defined in the Entity field
    dt.date(2025, 1, 1),   # Initial date of the query in format %Y-%m-%d
    dt.date(2025, 12, 31)  # Final date of the query in format %Y-%m-%d
)

In [38]:
pofe

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,2QEK,110.35500,110.35500,110.35500,110.35500,110.35500,110.35500,110.35500,110.35500,...,110.35500,110.35500,110.35500,110.35500,110.35500,110.35500,110.35500,110.35500,110.35500,2025-01-01
1,Recurso,3DDT,100.90002,100.90002,100.90002,100.90002,100.90002,100.90002,100.90002,100.90002,...,100.90002,100.90002,100.90002,100.90002,100.90002,100.90002,100.90002,100.90002,100.90002,2025-01-01
2,Recurso,3ENA,505.38900,505.38900,505.38900,505.38900,505.38900,505.38900,505.38900,505.38900,...,505.38900,505.38900,505.38900,505.38900,505.38900,505.38900,505.38900,505.38900,505.38900,2025-01-01
3,Recurso,3HF5,100.90003,100.90003,100.90003,100.90003,100.90003,100.90003,100.90003,100.90003,...,100.90003,100.90003,100.90003,100.90003,100.90003,100.90003,100.90003,100.90003,100.90003,2025-01-01
4,Recurso,3INX,98.00000,98.00000,98.00000,98.00000,98.00000,98.00000,98.00000,98.00000,...,98.00000,98.00000,98.00000,98.00000,98.00000,98.00000,98.00000,98.00000,98.00000,2025-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28991,Recurso,URA1,315.00000,315.00000,315.00000,315.00000,315.00000,315.00000,315.00000,315.00000,...,315.00000,315.00000,315.00000,315.00000,315.00000,315.00000,315.00000,315.00000,315.00000,2025-12-31
28992,Recurso,ZPA2,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,...,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,369.00000,2025-12-31
28993,Recurso,ZPA3,338.00000,338.00000,338.00000,338.00000,338.00000,338.00000,338.00000,338.00000,...,338.00000,338.00000,338.00000,338.00000,338.00000,338.00000,338.00000,338.00000,338.00000,2025-12-31
28994,Recurso,ZPA4,354.00000,354.00000,354.00000,354.00000,354.00000,354.00000,354.00000,354.00000,...,354.00000,354.00000,354.00000,354.00000,354.00000,354.00000,354.00000,354.00000,354.00000,2025-12-31


In [39]:
# Query generation data
pbna = objetoAPI.request_data(
    "PrecBolsNaci",        # Specify the name of the metric as defined in the MetricID field
    "Sistema",             # Specify the name of the entity as defined in the Entity field
    dt.date(2025, 1, 1),   # Initial date of the query in format %Y-%m-%d
    dt.date(2025, 12, 31)  # Final date of the query in format %Y-%m-%d
)

In [40]:
pbna

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,202.27695,202.27695,202.27695,202.27695,202.27695,202.27695,202.27695,113.12995,...,202.27695,202.27695,202.27695,256.44995,448.64795,448.64795,448.64795,242.42495,228.77695,2025-01-01
1,Sistema,Sistema,440.17379,253.84679,440.17379,253.84679,253.84679,440.17379,440.17379,440.17379,...,501.29679,501.29679,501.29679,620.17379,620.17379,620.17379,620.17379,501.29679,501.29679,2025-01-02
2,Sistema,Sistema,480.41457,480.41457,480.41457,480.41457,480.41457,480.41457,480.41457,480.41457,...,550.41457,550.41457,550.41457,550.41457,550.41457,550.41457,550.41457,550.41457,505.80357,2025-01-03
3,Sistema,Sistema,399.76688,399.76688,399.76688,399.76688,399.76688,399.76688,399.76688,399.76688,...,417.46988,447.88988,447.88988,450.76688,450.76688,450.76688,450.76688,447.88988,417.46988,2025-01-04
4,Sistema,Sistema,115.06702,125.55302,125.55302,125.55302,125.55302,125.55302,125.55302,125.55302,...,125.55302,125.55302,125.55302,401.55302,401.55302,401.55302,401.55302,237.55302,125.55302,2025-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Sistema,Sistema,263.85620,258.85620,258.85620,258.85620,258.85620,263.85620,258.85620,239.10720,...,263.85620,288.85620,338.85620,403.85620,403.85620,343.85620,338.85620,338.85620,338.85620,2025-12-27
361,Sistema,Sistema,286.86679,286.86679,286.86679,286.86679,286.86679,286.86679,286.86679,251.86679,...,286.86679,286.86679,331.86679,336.86679,336.86679,336.86679,331.86679,331.86679,331.86679,2025-12-28
362,Sistema,Sistema,306.76024,306.76024,306.76024,306.76024,306.76024,306.76024,306.76024,306.76024,...,330.76124,330.76124,356.76024,356.76024,356.76024,356.76024,356.76024,356.76024,330.76124,2025-12-29
363,Sistema,Sistema,306.58690,280.58590,280.58590,280.58590,280.58590,280.58590,306.58690,280.58590,...,306.58690,306.58690,325.58590,326.58590,326.58590,326.58590,326.58590,325.58590,306.58690,2025-12-30


In [41]:
# Query generation data
promcont = objetoAPI.request_data(
    "PrecPromCont",        # Specify the name of the metric as defined in the MetricID field
    "Sistema",             # Specify the name of the entity as defined in the Entity field
    dt.date(2025, 1, 1),   # Initial date of the query in format %Y-%m-%d
    dt.date(2025, 12, 31)  # Final date of the query in format %Y-%m-%d
)

In [42]:
promcont

,Id,Value,Date
0,Sistema,299.50148,2025-01-01
1,Sistema,307.96498,2025-01-02
2,Sistema,309.61714,2025-01-03
3,Sistema,306.01161,2025-01-04
4,Sistema,298.79399,2025-01-05
...,...,...,...
360,Sistema,295.07359,2025-12-27
361,Sistema,295.81525,2025-12-28
362,Sistema,296.28467,2025-12-29
363,Sistema,295.70284,2025-12-30


In [43]:
# Query generation data
gene = objetoAPI.request_data(
    "Gene",        # Specify the name of the metric as defined in the MetricID field
    "Recurso",             # Specify the name of the entity as defined in the Entity field
    dt.date(2025, 1, 1),   # Initial date of the query in format %Y-%m-%d
    dt.date(2025, 12, 31)  # Final date of the query in format %Y-%m-%d
)

Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\mrozo\Apps\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\mrozo\Apps\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\mrozo\Apps\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\mrozo\Apps\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
Co

In [44]:
gene

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,2QBW,19287.52,19323.73,19319.52,19321.37,19318.58,19306.80,19301.65,19302.39,...,19261.85,19266.97,19273.35,19260.71,19321.24,19138.46,18833.77,18702.22,19241.29,2025-01-01
1,Recurso,2QRL,652.23,652.11,651.99,652.17,652.08,652.05,651.90,651.90,...,651.75,651.78,651.78,651.72,651.69,651.60,651.66,651.81,651.66,2025-01-01
2,Recurso,2QV2,17600.40,16642.80,16221.60,15678.00,14738.40,14634.00,14630.40,14007.60,...,12711.60,12715.20,11797.20,11746.80,11746.80,11948.40,12704.40,7034.40,8676.00,2025-01-01
3,Recurso,2S6Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.40,52.80,111.60,0.60,2025-01-01
4,Recurso,2S6S,7809.60,43.20,8702.40,9129.60,9189.60,9158.40,9180.00,1058.40,...,NaN,NaN,6031.20,8100.00,8659.20,9410.40,9045.60,9465.60,7226.40,2025-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122803,Recurso,TYP3,48845.86,49098.03,49078.87,49065.26,49198.37,49183.01,48866.39,48501.21,...,48617.89,48480.61,48353.75,48136.91,48352.86,48564.74,48790.02,48971.93,49130.83,2025-12-31
122804,Recurso,URA1,79491.20,79492.23,79486.91,79487.52,79488.76,79471.52,79489.95,79496.85,...,79472.11,78575.84,78615.75,140500.72,157904.36,158836.50,154528.71,120141.99,97931.40,2025-12-31
122805,Recurso,USQ1,776.08,721.14,704.99,698.52,708.60,752.35,874.90,990.80,...,1125.22,1103.80,1078.23,1046.51,1020.21,959.53,883.51,787.96,749.20,2025-12-31
122806,Recurso,VLT1,8791.20,8639.40,8483.20,8320.40,8217.00,8892.40,8775.80,8610.80,...,8681.20,9031.00,8824.20,8786.80,8833.00,8837.40,8800.00,8683.40,8514.00,2025-12-31


In [45]:
# Query generation data
sources = objetoAPI.request_data(
    "ListadoRecursos",        # Specify the name of the metric as defined in the MetricID field
    "Sistema",             # Specify the name of the entity as defined in the Entity field
    dt.date(2025, 1, 1),   # Initial date of the query in format %Y-%m-%d
    dt.date(2025, 12, 31)  # Final date of the query in format %Y-%m-%d
)

In [46]:
sources

,Id,Values_Code,Values_Name,Values_Type,Values_Disp,Values_RecType,Values_CompanyCode,Values_EnerSource,Values_OperStartdate,Values_State,Date
0,Sistema,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ISGG,AGUA,2014-03-31,OPERACION,2026-01-17
1,Sistema,2QEK,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,2026-01-17
2,Sistema,2QRL,LA REBUSCA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,GEN. DISTRIBUIDA,HZEG,AGUA,2014-07-24,OPERACION,2026-01-17
3,Sistema,2QV2,BAJO TULUA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,EPSG,AGUA,2015-01-30,OPERACION,2026-01-17
4,Sistema,2R22,LAGUNETA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ENDG,AGUA,2014-12-17,OPERACION,2026-01-17
...,...,...,...,...,...,...,...,...,...,...,...
1436,Sistema,VNTB,VENTANA B,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,HDTG,AGUA,1957-11-01,OPERACION,2026-01-17
1437,Sistema,ZPA2,ZIPAEMG 2,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1964-01-01,OPERACION,2026-01-17
1438,Sistema,ZPA3,ZIPAEMG 3,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1976-01-01,OPERACION,2026-01-17
1439,Sistema,ZPA4,ZIPAEMG 4,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1981-04-01,OPERACION,2026-01-17


In [49]:
# Define the list of DataFrames you want to export
dfs = [pofe, pbna, promcont, gene, sources]  # Replace with your actual DataFrames
names = ["bid_price", "spot_price", "secondarymarket_price", "generation", "source_descrip"]  # Names for each Excel file

# Define the target folder path
target_folder = r"C:\Users\mrozo\OneDrive\Documentos\GitHub\EnergyPrices-PythonTableauProject\Datasets"

# Loop through DataFrames and save each one as an Excel file
for df, name in zip(dfs, names):
    file_path = f"{target_folder}\\{name}.xlsx"  # Build full file path
    df.to_excel(file_path, index=False)          # Save DataFrame without index
    print(f"Saved: {file_path}")                 # Print confirmation message

Saved: C:\Users\mrozo\OneDrive\Documentos\GitHub\EnergyPrices-PythonTableauProject\Datasets\bid_price.xlsx
Saved: C:\Users\mrozo\OneDrive\Documentos\GitHub\EnergyPrices-PythonTableauProject\Datasets\spot_price.xlsx
Saved: C:\Users\mrozo\OneDrive\Documentos\GitHub\EnergyPrices-PythonTableauProject\Datasets\secondarymarket_price.xlsx
Saved: C:\Users\mrozo\OneDrive\Documentos\GitHub\EnergyPrices-PythonTableauProject\Datasets\generation.xlsx
Saved: C:\Users\mrozo\OneDrive\Documentos\GitHub\EnergyPrices-PythonTableauProject\Datasets\source_descrip.xlsx


<a id='section3'></a>
# 3. Data Processing in Formats 

Data processing requires reading the latest version of the datasets in order to generate the tables and charts that will serve as inputs for the final product where all results will be published. The first step is **data cleaning and processing**.
For this exercise, we assume that we will only work with centrally dispatched plants, since these participate proactively in the market through bid prices. Additionally, all tables have different column names and are currently in wide format. 

### Tasks to perform: 
- **Rename columns** to ensure consistency across all datasets
- **Reshape tables into long format** for easier analysis and visualization
- **Merge datasets** into a unified base for processing }
- **Organize and filter values** according to defined conditions

# Renaming columns

In [50]:
# Loop through each DataFrame and print its column names
for i, df in enumerate(dfs, start=1):
    print(f"DataFrame {i} columns:")
    print(df.columns.tolist())  # Convert Index object to a Python list
    print("-" * 40)  # Separator line for readability

DataFrame 1 columns:
['Id', 'Values_code', 'Values_Hour01', 'Values_Hour02', 'Values_Hour03', 'Values_Hour04', 'Values_Hour05', 'Values_Hour06', 'Values_Hour07', 'Values_Hour08', 'Values_Hour09', 'Values_Hour10', 'Values_Hour11', 'Values_Hour12', 'Values_Hour13', 'Values_Hour14', 'Values_Hour15', 'Values_Hour16', 'Values_Hour17', 'Values_Hour18', 'Values_Hour19', 'Values_Hour20', 'Values_Hour21', 'Values_Hour22', 'Values_Hour23', 'Values_Hour24', 'Date']
----------------------------------------
DataFrame 2 columns:
['Id', 'Values_code', 'Values_Hour01', 'Values_Hour02', 'Values_Hour03', 'Values_Hour04', 'Values_Hour05', 'Values_Hour06', 'Values_Hour07', 'Values_Hour08', 'Values_Hour09', 'Values_Hour10', 'Values_Hour11', 'Values_Hour12', 'Values_Hour13', 'Values_Hour14', 'Values_Hour15', 'Values_Hour16', 'Values_Hour17', 'Values_Hour18', 'Values_Hour19', 'Values_Hour20', 'Values_Hour21', 'Values_Hour22', 'Values_Hour23', 'Values_Hour24', 'Date']
----------------------------------------


In [59]:
# Define the dictionary with the new column names (applies to all DataFrames)
rename_map = {
    "Values_code": "Plant", 
    "Values_Code": "Plant",  
    "Values_Name": "PlantName", 
    "Values_Type": "PlantType",              
    "Values_Disp": "PlantDispType", 
    "Values_RecType": "PlantRecType",
    "Values_CompanyCode": "PlantCompany", 
    "Values_EnerSource": "PlantEnerSource",
    "Values_OperStartdate": "PlantOperStartDate", 
    "Values_State": "PlantState"
}

# Loop through all DataFrames and apply the renaming rules
for i, df in enumerate(dfs, start=1):
    df.rename(columns=rename_map, inplace=True)  # Rename columns if they exist
    print(f"DataFrame {i} renamed columns:", df.columns.tolist())  # Show updated column names

DataFrame 1 renamed columns: ['Id', 'Plant', 'Values_Hour01', 'Values_Hour02', 'Values_Hour03', 'Values_Hour04', 'Values_Hour05', 'Values_Hour06', 'Values_Hour07', 'Values_Hour08', 'Values_Hour09', 'Values_Hour10', 'Values_Hour11', 'Values_Hour12', 'Values_Hour13', 'Values_Hour14', 'Values_Hour15', 'Values_Hour16', 'Values_Hour17', 'Values_Hour18', 'Values_Hour19', 'Values_Hour20', 'Values_Hour21', 'Values_Hour22', 'Values_Hour23', 'Values_Hour24', 'Date']
DataFrame 2 renamed columns: ['Id', 'Plant', 'Values_Hour01', 'Values_Hour02', 'Values_Hour03', 'Values_Hour04', 'Values_Hour05', 'Values_Hour06', 'Values_Hour07', 'Values_Hour08', 'Values_Hour09', 'Values_Hour10', 'Values_Hour11', 'Values_Hour12', 'Values_Hour13', 'Values_Hour14', 'Values_Hour15', 'Values_Hour16', 'Values_Hour17', 'Values_Hour18', 'Values_Hour19', 'Values_Hour20', 'Values_Hour21', 'Values_Hour22', 'Values_Hour23', 'Values_Hour24', 'Date']
DataFrame 3 renamed columns: ['Id', 'Value', 'Date']
DataFrame 4 renamed colum

In [61]:
# Example
sources

,Id,Plant,PlantName,PlantType,PlantDispType,PlantRecType,PlantCompany,PlantEnerSource,PlantOperStartDate,PlantState,Date
0,Sistema,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ISGG,AGUA,2014-03-31,OPERACION,2026-01-17
1,Sistema,2QEK,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,2026-01-17
2,Sistema,2QRL,LA REBUSCA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,GEN. DISTRIBUIDA,HZEG,AGUA,2014-07-24,OPERACION,2026-01-17
3,Sistema,2QV2,BAJO TULUA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,EPSG,AGUA,2015-01-30,OPERACION,2026-01-17
4,Sistema,2R22,LAGUNETA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ENDG,AGUA,2014-12-17,OPERACION,2026-01-17
...,...,...,...,...,...,...,...,...,...,...,...
1436,Sistema,VNTB,VENTANA B,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,HDTG,AGUA,1957-11-01,OPERACION,2026-01-17
1437,Sistema,ZPA2,ZIPAEMG 2,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1964-01-01,OPERACION,2026-01-17
1438,Sistema,ZPA3,ZIPAEMG 3,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1976-01-01,OPERACION,2026-01-17
1439,Sistema,ZPA4,ZIPAEMG 4,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1981-04-01,OPERACION,2026-01-17


# Reshaping dfs

In [73]:
# Define the list of DataFrames and their names
dfs = [pofe, pbna, promcont, gene, sources]
names = ["bid_price", "spot_price", "secondarymarket_price", "generation", "source_descrip"]

reshaped_dfs = {}

for df, name in zip(dfs, names):
    hour_cols = [c for c in df.columns if c.startswith("Values_Hour")]
    
    if hour_cols:
        id_vars = [c for c in df.columns if c not in hour_cols]
        df_long = df.melt(id_vars=id_vars, value_vars=hour_cols,
                          var_name="Hour", value_name="Value")
        df_long["Hour"] = df_long["Hour"].str.replace("Values_Hour", "").astype(int) - 1
        df_long["Value"] = df_long["Value"].fillna(0)
        
        reshaped_dfs[f"{name}_reshape"] = df_long
        print(f"{name}_reshape created.")
    else:
        # Store the original DataFrame under its name if no reshape is needed
        reshaped_dfs[name] = df
        print(f"{name} kept without reshape.")

bid_price_reshape created.
spot_price_reshape created.
secondarymarket_price kept without reshape.
generation_reshape created.
source_descrip kept without reshape.


In [79]:
# Loop through all DataFrames in reshaped_dfs
for key in reshaped_dfs:
    if "Id" in reshaped_dfs[key].columns:
        # Drop the 'id' column if it exists
        reshaped_dfs[key] = reshaped_dfs[key].drop(columns=["Id"])
        print(f"Dropped 'id' column from {key}")
    else:
        print(f"No 'id' column found in {key}")

Dropped 'id' column from bid_price_reshape
Dropped 'id' column from spot_price_reshape
Dropped 'id' column from secondarymarket_price
Dropped 'id' column from generation_reshape
Dropped 'id' column from source_descrip


In [80]:
bid_price_reshape = reshaped_dfs["bid_price_reshape"]
spot_price_reshape = reshaped_dfs["spot_price_reshape"]
secondarymarket_price =reshaped_dfs["secondarymarket_price"]
generation_reshape = reshaped_dfs["generation_reshape"]
source_descrip = reshaped_dfs["source_descrip"]

In [82]:
# Rename the Value column in each DataFrame to its dataset name
bid_price_reshape = bid_price_reshape.rename(columns={"Value": "bid_price"})
spot_price_reshape = spot_price_reshape.rename(columns={"Value": "spot_price"})
secondarymarket_price = secondarymarket_price.rename(columns={"Value": "secondarymarket_price"})
generation_reshape = generation_reshape.rename(columns={"Value": "generation"})

# Merge datasets

In [130]:
# First, merge all the data related to the plants
# Drop Date column from source_descrip before merging 
source_descrip_clean = source_descrip.drop(columns=["Date"], errors="ignore")

# Merge bid_price_reshape with the cleaned source_descrip on Plant
plant_info = bid_price_reshape.merge(source_descrip_clean,on=["Plant"], how="inner")
plant_info = plant_info.merge(generation_reshape, on=["Date","Hour","Plant"], how="inner")                                

In [131]:
# Second, merge all the data related to the Date, Plants and Hour
market_info = spot_price_reshape.merge(secondarymarket_price,on=["Date"], how="inner") 

In [132]:
# Drop Date column from Plant before merging 
market_info = market_info.drop(columns=["Plant"], errors="ignore")

In [133]:
market_info

,Date,Hour,spot_price,secondarymarket_price
0,2025-01-01,0,202.27695,299.50148
1,2025-01-02,0,440.17379,307.96498
2,2025-01-03,0,480.41457,309.61714
3,2025-01-04,0,399.76688,306.01161
4,2025-01-05,0,115.06702,298.79399
...,...,...,...,...
8755,2025-12-27,23,338.85620,295.07359
8756,2025-12-28,23,331.86679,295.81525
8757,2025-12-29,23,330.76124,296.28467
8758,2025-12-30,23,306.58690,295.70284


In [134]:
# Third, merge all the previous data
plantmarket_info = plant_info.merge(market_info,on=["Date","Hour"], how="inner") 

In [135]:
plantmarket_info

,Plant,Date,Hour,bid_price,PlantName,PlantType,PlantDispType,PlantRecType,PlantCompany,PlantEnerSource,PlantOperStartDate,PlantState,generation,spot_price,secondarymarket_price
0,3DDT,2025-01-01,0,100.90002,LATAM SOLAR LA LOMA,SOLAR,DESPACHADO CENTRALMENTE,NORMAL,ENDG,RAD SOLAR,2024-06-24,OPERACION,0.00,202.27695,299.50148
1,3ENA,2025-01-01,0,505.38900,TESORITO,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,EPSG,GAS,2022-09-13,OPERACION,0.00,202.27695,299.50148
2,3HF5,2025-01-01,0,100.90003,FUNDACION,SOLAR,DESPACHADO CENTRALMENTE,NORMAL,ENDG,RAD SOLAR,2024-06-24,OPERACION,0.00,202.27695,299.50148
3,3INX,2025-01-01,0,98.00000,CARACOLI I,SOLAR,DESPACHADO CENTRALMENTE,NORMAL,SDCG,RAD SOLAR,2024-12-22,OPERACION,0.00,202.27695,299.50148
4,3IQA,2025-01-01,0,91.89100,SUNNORTE,SOLAR,DESPACHADO CENTRALMENTE,NORMAL,GNSG,RAD SOLAR,2024-11-26,OPERACION,0.00,202.27695,299.50148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444259,TSJ1,2025-12-31,23,236.90000,TASAJERO 1,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,TRMG,CARBON,1985-02-20,OPERACION,165627.10,270.45981,294.94318
444260,TSJ2,2025-12-31,23,232.90000,TASAJERO 2,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,TERG,CARBON,2015-11-30,OPERACION,170692.10,270.45981,294.94318
444261,TYP2,2025-12-31,23,90.00002,TERMOYOPAL 2,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,TYPG,GAS,2004-07-29,OPERACION,26696.38,270.45981,294.94318
444262,TYP3,2025-12-31,23,90.00000,TERMOYOPAL G3,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,TYPG,GAS,2020-08-31,OPERACION,49130.83,270.45981,294.94318


# Organize and filter data

In [136]:
# Filter rows where PlantDispType is "DESPACHADO CENTRALMENTE"
plantmarket_info_filtered = plantmarket_info[
    plantmarket_info["PlantDispType"] == "DESPACHADO CENTRALMENTE"
]

# Sort the DataFrame by Plant, Date, and Hour
plantmarket_info_filtered = plantmarket_info_filtered.sort_values(
    by=["Plant", "Date", "Hour"]
)

# Reset the index for clarity after sorting
plantmarket_info_filtered = plantmarket_info_filtered.reset_index(drop=True)

# Preview the first rows of the filtered and sorted DataFrame
print(plantmarket_info_filtered.head())

  Plant       Date  Hour  bid_price PlantName   PlantType  \
0  2QEK 2025-01-04     0    109.131  SALTO II  HIDRAULICA   
1  2QEK 2025-01-04     1    109.131  SALTO II  HIDRAULICA   
2  2QEK 2025-01-04     2    109.131  SALTO II  HIDRAULICA   
3  2QEK 2025-01-04     3    109.131  SALTO II  HIDRAULICA   
4  2QEK 2025-01-04     4    109.131  SALTO II  HIDRAULICA   

             PlantDispType  PlantRecType PlantCompany PlantEnerSource  \
0  DESPACHADO CENTRALMENTE  FILO DE AGUA         ENDG            AGUA   
1  DESPACHADO CENTRALMENTE  FILO DE AGUA         ENDG            AGUA   
2  DESPACHADO CENTRALMENTE  FILO DE AGUA         ENDG            AGUA   
3  DESPACHADO CENTRALMENTE  FILO DE AGUA         ENDG            AGUA   
4  DESPACHADO CENTRALMENTE  FILO DE AGUA         ENDG            AGUA   

  PlantOperStartDate PlantState  generation  spot_price  secondarymarket_price  
0         2014-06-25  OPERACION         0.0   399.76688              306.01161  
1         2014-06-25  OPERACION 

In [137]:
plantmarket_info_filtered

,Plant,Date,Hour,bid_price,PlantName,PlantType,PlantDispType,PlantRecType,PlantCompany,PlantEnerSource,PlantOperStartDate,PlantState,generation,spot_price,secondarymarket_price
0,2QEK,2025-01-04,0,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161
1,2QEK,2025-01-04,1,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161
2,2QEK,2025-01-04,2,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161
3,2QEK,2025-01-04,3,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161
4,2QEK,2025-01-04,4,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444259,ZPA5,2025-12-25,19,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,339.89084,293.66590
444260,ZPA5,2025-12-25,20,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,339.89084,293.66590
444261,ZPA5,2025-12-25,21,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,256.79084,293.66590
444262,ZPA5,2025-12-25,22,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,252.89084,293.66590


<a id='section4'></a> 
# 4. Object with Processing Results 

In this section, we will conduct an exercise aimed at understanding the trends between the energy spot market and the secondary energy market. The goal is to create key indicators and extract the data required for analysis in Tableau. Additionally, we want to evaluate which market (spot or secondary) may be more attractive in terms of profitability. To achieve this, we will create a profitability indicator defined as:


$$
\pi_{i,h} = \bigl(P_{\text{sale, COP/kWh}} - MC_{P\text{bid, COP/kWh}}\bigr)\cdot \mathrm{Generation}_{\text{kWh}}
$$
$
$



This indicator will allow us to analyze how profitability evolves across the markets. 

In reality, energy markets are far more complex; this formulation is a **simplified model** intended to isolate other variables for analysis. We will assume that, given the competitive nature of the market, the bid price is equal to the marginal cost.

By focusing on the relationship between spot prices, secondary market prices, and generation, we can better understand potential profitability trends while keeping the model tractable for visualization and exploration in Tablea.


In [138]:
plantmarket_info_filtered["profit_spot"]=(plantmarket_info_filtered["spot_price"]-plantmarket_info_filtered["bid_price"])*plantmarket_info_filtered["generation"]
plantmarket_info_filtered["profit_secondarymarket"]=(plantmarket_info_filtered["secondarymarket_price"]-plantmarket_info_filtered["bid_price"])*plantmarket_info_filtered["generation"]

In [139]:
# Create Datetime variables for Tableau
plantmarket_info_filtered["Datetime"] = pd.to_datetime(
    plantmarket_info_filtered["Date"].astype(str) + " " + plantmarket_info_filtered["Hour"].astype(str) + ":00",
    format="%Y-%m-%d %H:%M"
)

In [140]:
plantmarket_info_filtered

,Plant,Date,Hour,bid_price,PlantName,PlantType,PlantDispType,PlantRecType,PlantCompany,PlantEnerSource,PlantOperStartDate,PlantState,generation,spot_price,secondarymarket_price,profit_spot,profit_secondarymarket,Datetime
0,2QEK,2025-01-04,0,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161,0.0,0.0,2025-01-04 00:00:00
1,2QEK,2025-01-04,1,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161,0.0,0.0,2025-01-04 01:00:00
2,2QEK,2025-01-04,2,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161,0.0,0.0,2025-01-04 02:00:00
3,2QEK,2025-01-04,3,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161,0.0,0.0,2025-01-04 03:00:00
4,2QEK,2025-01-04,4,109.131,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION,0.0,399.76688,306.01161,0.0,0.0,2025-01-04 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444259,ZPA5,2025-12-25,19,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,339.89084,293.66590,0.0,-0.0,2025-12-25 19:00:00
444260,ZPA5,2025-12-25,20,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,339.89084,293.66590,0.0,-0.0,2025-12-25 20:00:00
444261,ZPA5,2025-12-25,21,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,256.79084,293.66590,-0.0,-0.0,2025-12-25 21:00:00
444262,ZPA5,2025-12-25,22,329.000,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,0.0,252.89084,293.66590,-0.0,-0.0,2025-12-25 22:00:00


In [141]:
# Create Datetime variables for Tableau
market_info["Datetime"] = pd.to_datetime(
    market_info["Date"].astype(str) + " " + market_info["Hour"].astype(str) + ":00",
    format="%Y-%m-%d %H:%M"
)

In [142]:
market_info

,Date,Hour,spot_price,secondarymarket_price,Datetime
0,2025-01-01,0,202.27695,299.50148,2025-01-01 00:00:00
1,2025-01-02,0,440.17379,307.96498,2025-01-02 00:00:00
2,2025-01-03,0,480.41457,309.61714,2025-01-03 00:00:00
3,2025-01-04,0,399.76688,306.01161,2025-01-04 00:00:00
4,2025-01-05,0,115.06702,298.79399,2025-01-05 00:00:00
...,...,...,...,...,...
8755,2025-12-27,23,338.85620,295.07359,2025-12-27 23:00:00
8756,2025-12-28,23,331.86679,295.81525,2025-12-28 23:00:00
8757,2025-12-29,23,330.76124,296.28467,2025-12-29 23:00:00
8758,2025-12-30,23,306.58690,295.70284,2025-12-30 23:00:00


In [143]:
# Export each DataFrame to its own Excel file
market_info.to_excel("energymarket_trends.xlsx", index=False)
plantmarket_info_filtered.to_excel("plantprofit_info.xlsx", index=False)